In [3]:
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

## [2 pts.] Utilizando el dataset disponible realizar la estructuración adecuada para generar un dataframe/archivo donde la primera columna sea el texto (datos) y la segunda columna sea la etiqueta respectiva (Considerar únicamente etiquetas Positivo y Negativo).

In [5]:
#Data para entrenar
datatrain = pd.read_csv('Corona_NLP_train.csv',delimiter = ',',header = 0,encoding='8859') 
datatrain

FileNotFoundError: [Errno 2] No such file or directory: 'Corona_NLP_train.csv'

In [ ]:
#Data para test
datatest = pd.read_csv('Corona_NLP_train.csv'',delimiter = ',',header = 0,encoding='8859') 
datatest

In [ ]:
list(datatrain.values[:,-1]).count('Positive') +list(datatest.values[:,-1]).count('Negative')

In [5]:
data_total = pd.concat([datatrain, datatest], axis=0)
data_total

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
3793,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative
3795,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Extremely Negative


In [6]:
sentiment = []
tweets = []
for i in data_total.values:
    if ( i[-1] == 'Positive' or i[-1] == 'Negative'):
        sentiment.append(i[-1])
        tweets.append(i[4])

In [7]:
new_data = pd.DataFrame({'tweets':tweets,'sentiments':sentiment})
new_data

,tweets,sentiments
0,advice Talk to your neighbours family to excha...,Positive
1,Coronavirus Australia: Woolworths to give elde...,Positive
2,My food stock is not the only one which is emp...,Positive
3,As news of the regionÂs first confirmed COVID...,Positive
4,Cashier at grocery store was sharing his insig...,Positive
...,...,...
23322,Just been through K?piti New World which is bu...,Positive
23323,"Even though the Law Library is closed, ALL sub...",Positive
23324,"@RicePolitics @MDCounties Craig, will you call...",Negative
23325,Meanwhile In A Supermarket in Israel -- People...,Positive


## [2 pts.] Implementar un procedimiento (básico) de limpieza a la data obtenida

In [9]:
tweetpre = []
for i in new_data.values:
    cad = re.findall('[@#A-Za-z]+',i[0])
    w = [w.lower() for w in cad if len(w)>3]
    if(len(w) != 0):
        tweetpre.append(w)

In [10]:
datapre = pd.DataFrame({'tweets':tweetpre,'sentiments':sentiment})
datapre

,tweets,sentiments
0,"[advice, talk, your, neighbours, family, excha...",Positive
1,"[coronavirus, australia, woolworths, give, eld...",Positive
2,"[food, stock, only, which, empty, please, pani...",Positive
3,"[news, region, first, confirmed, covid, case, ...",Positive
4,"[cashier, grocery, store, sharing, insights, #...",Positive
...,...,...
23322,"[just, been, through, piti, world, which, busy...",Positive
23323,"[even, though, library, closed, subscribers, a...",Positive
23324,"[@ricepolitics, @mdcounties, craig, will, call...",Negative
23325,"[meanwhile, supermarket, israel, people, dance...",Positive


## [4 pts.] Obtener representaciones vectoriales de cada documento (tweet), mediante el siguiente procedimiento:

### A. Obtener las top-20 palabras más repetidas de cada clase. En total tendríamos 40 palabras top (20 representando positivos y 20 representado negativos). Estas palabras representarán nuestro universo de palabras características de tamaño 40:

### U = [w1, w2, w3, ..., w40]

In [58]:
#10000 primeros
totalwords1 = []
totalwords2 = []
for i in datapre.values:
    if(i[-1] == 'Positive'):
        totalwords1.extend(i[0])
    else:
        totalwords2.extend(i[0])

In [59]:
uniquewords1 = list(set(totalwords1))
uniquewords2 = list(set(totalwords2))

In [60]:
dic1 = {}
for w in uniquewords1:
    dic1[w] = totalwords1.count(w)
dic1

{'nebraska': 2,
 'fetching': 1,
 'subsidiaries': 1,
 'dimwits': 1,
 'malay': 1,
 '@afyarekod': 1,
 '#gartnersc': 1,
 'hlkx': 1,
 'vigilant': 18,
 'ytsdmva': 1,
 '#poopchallenge': 1,
 'arizona': 5,
 'claim': 23,
 'evading': 1,
 'dlec': 1,
 '#wearablebands': 1,
 'richardson': 1,
 'hotdogs': 1,
 'kisses': 1,
 'ofafsva': 1,
 'viable': 4,
 'conversant': 1,
 '@juliusagwu': 1,
 'lorries': 1,
 'zoumg': 1,
 '#grocerapp': 1,
 '#lifeskills': 1,
 '#shoplocal': 22,
 '#coronaireland': 2,
 '@daniellejade': 1,
 'intervene': 4,
 'shrink': 1,
 'freeport': 1,
 'cxxgrtsr': 1,
 '@seaworld': 1,
 'prevails': 2,
 'babysitter': 1,
 'improvise': 2,
 'lens': 1,
 '#kano': 1,
 'cnas': 1,
 'moderated': 1,
 'uyccbf': 1,
 'boot': 1,
 'fatalistically': 1,
 'enigym': 1,
 'sijnh': 1,
 'distilling': 6,
 'schedules': 1,
 'midst': 15,
 '@cppunecity': 2,
 'gzladyqicm': 1,
 'blockchain': 5,
 '#companywatch': 1,
 'downsizing': 2,
 'qwrnvno': 1,
 'evvb': 1,
 'yrbudn': 1,
 '#physics': 1,
 '#qantas': 2,
 'phoenix': 6,
 '#notthat

In [61]:
dic2 = {}
for w in uniquewords:
    dic2[w] = totalwords2.count(w)
dic2

{'bitching': 3,
 'nebraska': 3,
 'bykccar': 1,
 'fetching': 0,
 'viralism': 1,
 'subsidiaries': 1,
 'milkmen': 1,
 'tempted': 2,
 'misreported': 1,
 'dimwits': 1,
 'malay': 0,
 'hgyjg': 1,
 '@afyarekod': 0,
 '#gartnersc': 0,
 '@bannerhealth': 1,
 'hlkx': 0,
 'journo': 1,
 'latch': 1,
 '#financialinstitutions': 1,
 'vigilant': 6,
 'ytsdmva': 0,
 '#poopchallenge': 1,
 'latam': 1,
 'arizona': 9,
 '@natashakorecki': 1,
 'claim': 21,
 'evading': 1,
 '@ferretscot': 1,
 'hubbie': 1,
 'ufbbmp': 1,
 '#coronashutdown': 1,
 'dlec': 0,
 'ftopx': 1,
 '#primeday': 2,
 '#wearablebands': 0,
 'hairs': 1,
 'richardson': 1,
 'hotdogs': 0,
 'kisses': 1,
 'ofafsva': 0,
 'viable': 5,
 'conversant': 0,
 '@juliusagwu': 0,
 'lorries': 0,
 'zoumg': 0,
 '#grocerapp': 0,
 '#lifeskills': 0,
 '#shoplocal': 4,
 '#coronaireland': 0,
 '@daniellejade': 0,
 '#parecetomol': 1,
 'ahora': 1,
 'intervene': 6,
 'shrink': 4,
 'aerated': 1,
 'freeport': 0,
 'cxxgrtsr': 0,
 '@seaworld': 0,
 'prevails': 0,
 '#ussenate': 1,
 'bab

In [64]:
sort_d1 = sorted(dic1.items(), key=lambda x: x[1], reverse=True)
top20_1 = sort_d1[0:20]
sort_d2 = sorted(dic2.items(), key=lambda x: x[1], reverse=True)
top20_2 = sort_d2[0:20]

In [81]:
U = []
for i in top20_1:
    U.append(i[0])
for j in top20_2:
    U.append(j[0])

In [83]:
len(U)

40

### B. Para cada documento (tweet):
### Inicializar un vector o lista (V) de tamaño 40 con valores 0.
###  Analizar palabra por palabra:
###         Si la palabra se encuentra en el vector universo U, obtener la posición (POS) de la palabra en ese vector
###         En el vector V, incrementar en 1 el valor de la posición (POS).

In [84]:
Vector = []
for l in datapre.values:
    aux = []
    for i in U:
        if i in l[0]:
            a = l[0].count(i)
            aux.append(a)
        else:
            aux.append(0)
    Vector.append(aux)

In [86]:
len(Vector)

23327

### [5 pts.] En base a los vectores obtenidos en el paso anterior, implementar un proceso de clasificación (KNN) y obtener el accuracy respectivo en el conjunto de test.

In [89]:
X = Vector
Y = sentiment
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=10)

In [90]:
k = 1
a = [0]
f = False
while(f == False):
    
    clasificador = KNeighborsClassifier(n_neighbors=k)

    clasificador.fit(X_train,Y_train)

    Y_pred = clasificador.predict(X_test)

    mtc = accuracy_score(Y_test, Y_pred)
    
    if (mtc>a[k-1]):
        a.append(mtc)
    else:
        f = True
    k = k + 1

print('El mejor k es:',k)
print('El accuracy es: ',a[-1])

El mejor k es: 3
El accuracy es:  0.5167166738105443


### [7 pts.] Implementar un proceso de clustering (k=2) y obtener la frecuencia de las palabras de cada cluster. Reportar únicamente el top-10 de las palabras más repetidas de cada cluster.

In [91]:
modelo = KMeans(n_clusters=2)
modelo.fit(X)
out = modelo.predict(X)

In [101]:
m = pd.DataFrame(X)
m['out'] = out

total1 = []
total2 = []
for i in m.values:
    if(i[-1]==0):
        total1.extend(i[0])
    else:
        total2.extend(i[0])

0


TypeError: 'numpy.int64' object is not iterable